## Granum Feature Summary (CSV Load)

This notebook loads `data/churn_data_landscaping.csv` using a **repo-friendly relative path** (works after pushing to GitHub and cloning).

- Expected kernel: **Python (data_viz_env)**
- CSV location: `data/churn_data_landscaping.csv`


### 1) Imports + path configuration


In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
from IPython.display import display
from scipy.stats import f_oneway


In [15]:
# Repo-friendly relative path (assumes notebook is run from the repo root).
# If you run from another working directory, we fall back to searching upward.
CSV_RELATIVE_PATH = Path("data") / "churn_data_landscaping.csv"


def find_csv(start_dir: Path) -> Path:
    """Find the CSV by walking up directories from start_dir."""
    start_dir = start_dir.resolve()
    for current in [start_dir, *start_dir.parents]:
        candidate = current / CSV_RELATIVE_PATH
        if candidate.exists():
            return candidate
    raise FileNotFoundError(
        f"Could not find {CSV_RELATIVE_PATH.as_posix()} starting from {start_dir}. "
        "Run this notebook from the repo root or move it back into the repo."
    )


CSV_PATH = find_csv(Path.cwd())
CSV_PATH


WindowsPath('C:/Users/kaedi/Documents/Python/Portfolio/projects/landscaping_saas_churn/data/churn_data_landscaping.csv')

### 2) Load the CSV


In [16]:
df = pd.read_csv(CSV_PATH)

# Small cleanliness tweaks (safe to do, keeps display tidy)
df.columns = df.columns.str.strip()
if "Feature" in df.columns:
    df["Feature"] = df["Feature"].astype(str).str.strip()

df.head()


,CASE ID,LicenseCount,AccountStatus,ChurnFlag,Activation Date Range,Tenure,Cancellation Date,ARR Category,Persona,QuickbooksComplete,BudgetCount,ActiveBudgetCount,StandardEstimateCount,ServicedEstimateCount,LeadPortalActiveCount,TimesheetCount,InvoiceCount,OnlinePaymentCountThisYear,OnlinePaymentValueThisYear,TimesheetPhotos Count
0,26,18,Active,0,10/10/2013,148,NaN,SMB,P2,1,22,1,3017,1935,135,12417,1,0,0.0,1
1,27,1,Active,0,10/10/2013,148,NaN,SMB,P3,1,27,2,579,42,5,5569,0,0,0.0,578
2,28,8,Active,0,10/10/2013,148,NaN,SMB,P2,1,41,14,2341,1112,8,11318,0,0,0.0,1879
3,29,15,Active,0,10/10/2013,148,NaN,SMB,P2,1,33,10,408,909,208,9307,0,0,0.0,1485
4,35,5,Active,0,10/10/2013,148,NaN,SMB,P1,1,20,3,1241,7,0,4094,0,0,0.0,0


### 3) Nicely formatted view


In [ ]:
# display(
#     df.sort_values("Rank")
#     .style
#     .format(
#         {
#             "Active Mean": "{:.3f}",
#             "Churn Mean": "{:.3f}",
#             "Mean Diff": "{:.3f}",
#             "p value": "{:.4f}",
#         }
#     )
# )


### 4) Quick checks


In [18]:
df.shape, df.columns.tolist()


((4358, 20),
 ['CASE ID',
  'LicenseCount',
  'AccountStatus',
  'ChurnFlag',
  'Activation Date Range',
  'Tenure',
  'Cancellation Date',
  'ARR Category',
  'Persona',
  'QuickbooksComplete',
  'BudgetCount',
  'ActiveBudgetCount',
  'StandardEstimateCount',
  'ServicedEstimateCount',
  'LeadPortalActiveCount',
  'TimesheetCount',
  'InvoiceCount',
  'OnlinePaymentCountThisYear',
  'OnlinePaymentValueThisYear',
  'TimesheetPhotos Count'])

## F-test ranking (AccountStatus: Active vs Churned)

This section loads the account-level dataset (`data/churn_data_landscaping.csv`) and runs a **one-way ANOVA F-test** for each numeric metric to quantify association with `AccountStatus`.

- **Grouping**: `Active` vs **everything else** (mapped to `Churned`)
- **Ranking**: highest F-statistic (and lowest p-value) first


In [19]:
# Load the account-level churn dataset
CHURN_CSV_RELATIVE_PATH = Path("data") / "churn_data_landscaping.csv"


def find_file(start_dir: Path, relative_path: Path) -> Path:
    """Find a repo file by walking up directories from start_dir."""
    start_dir = start_dir.resolve()
    for current in [start_dir, *start_dir.parents]:
        candidate = current / relative_path
        if candidate.exists():
            return candidate
    raise FileNotFoundError(
        f"Could not find {relative_path.as_posix()} starting from {start_dir}. "
        "Run this notebook from the repo root or move it back into the repo."
    )


CHURN_CSV_PATH = find_file(Path.cwd(), CHURN_CSV_RELATIVE_PATH)
CHURN_CSV_PATH


WindowsPath('C:/Users/kaedi/Documents/Python/Portfolio/projects/landscaping_saas_churn/data/churn_data_landscaping.csv')

In [20]:
accounts_df = pd.read_csv(CHURN_CSV_PATH)

# Standardize column names a bit for safer selection
accounts_df.columns = accounts_df.columns.astype(str).str.strip()

# Normalize AccountStatus into a clean binary grouping
status_col = "AccountStatus"
if status_col not in accounts_df.columns:
    raise KeyError(f"Expected '{status_col}' column in {CHURN_CSV_RELATIVE_PATH.as_posix()}")

accounts_df[status_col] = accounts_df[status_col].astype(str).str.strip()
accounts_df["AccountStatusBinary"] = np.where(
    accounts_df[status_col].str.lower().eq("active"),
    "Active",
    "Churned",
)

accounts_df[[status_col, "AccountStatusBinary"]].value_counts().rename("count")


AccountStatus  AccountStatusBinary
Active         Active                 3997
Churn          Churned                 361
Name: count, dtype: int64

In [ ]:
def f_test_rank_metrics(
    df: pd.DataFrame,
    group_col: str = "AccountStatusBinary",
    exclude_cols: set[str] | None = None,
    min_n_per_group: int = 20,
) -> pd.DataFrame:
    """Run 1-way ANOVA F-test per numeric column; return ranked results."""
    if exclude_cols is None:
        exclude_cols = set()

    # Candidate metric columns = numeric columns minus exclusions
    numeric_cols = df.select_dtypes(include=["number"]).columns
    metric_cols = [c for c in numeric_cols if c not in exclude_cols]

    groups = ["Active", "Churned"]
    results: list[dict] = []

    for col in metric_cols:
        a = df.loc[df[group_col] == groups[0], col].dropna()
        b = df.loc[df[group_col] == groups[1], col].dropna()

        if len(a) < min_n_per_group or len(b) < min_n_per_group:
            continue

        f_stat, p_val = f_oneway(a.to_numpy(), b.to_numpy())

        mean_a = float(a.mean())
        mean_b = float(b.mean())
        diff = mean_a - mean_b
        pct_diff = diff / mean_a if mean_a != 0 else np.nan

        results.append(
            {
                "metric": col,
                "n_active": int(len(a)),
                "n_churned": int(len(b)),
                "mean_active": mean_a,
                "mean_churned": mean_b,
                "diff_active_minus_churned": diff,
                "pct_diff_vs_active": pct_diff,
                "F": float(f_stat),
                "p_value": float(p_val),
            }
        )

    out = pd.DataFrame(results)
    if out.empty:
        return out

    return out.sort_values(["F", "p_value"], ascending=[False, True]).reset_index(drop=True)


# Exclude obvious non-metric numeric fields
exclude = {
    "CASE ID",  # identifier
    "ChurnFlag",  # target/label
}

rank_df = f_test_rank_metrics(accounts_df, exclude_cols=exclude, min_n_per_group=20)
rank_df.head(15)


,metric,n_active,n_churned,mean_active,mean_churned,diff_active_minus_churned,pct_diff_vs_active,F,p_value
0,LicenseCount,3997,361,13.151614,4.238227,8.913387,0.677741,80.645923,3.939814e-19
1,LeadPortalActiveCount,3997,361,173.962722,71.490305,102.472417,0.589048,30.115405,4.301925e-08
2,StandardEstimateCount,3997,361,1090.374031,547.506925,542.867105,0.497872,18.325317,1.902209e-05
3,ServicedEstimateCount,3997,361,590.137853,235.279778,354.858075,0.601314,16.561677,4.792743e-05
4,OnlinePaymentCountThisYear,3997,361,49.108081,3.711911,45.396170,0.924413,16.381361,5.269063e-05
5,BudgetCount,3997,361,8.417813,6.418283,1.999531,0.237536,16.006860,6.416115e-05
6,OnlinePaymentValueThisYear,3997,361,39496.134789,1485.673130,38010.461658,0.962384,13.912607,1.939464e-04
7,TimesheetCount,3997,361,4754.244433,2577.473684,2176.770749,0.457858,13.394766,2.553165e-04
8,InvoiceCount,3997,361,1455.899174,568.590028,887.309147,0.609458,12.069676,5.174788e-04
9,ActiveBudgetCount,3997,361,3.268451,3.094183,0.174269,0.053318,1.076982,2.994313e-01


In [22]:
if rank_df.empty:
    display(rank_df)
else:
    display(
        rank_df.head(30)
        .style
        .format(
            {
                "mean_active": "{:.3f}",
                "mean_churned": "{:.3f}",
                "diff_active_minus_churned": "{:.3f}",
                "pct_diff_vs_active": "{:.2%}",
                "F": "{:.3f}",
                "p_value": "{:.3e}",
            }
        )
    )


,metric,n_active,n_churned,mean_active,mean_churned,diff_active_minus_churned,pct_diff_vs_active,F,p_value
0,LicenseCount,3997,361,13.152,4.238,8.913,67.77%,80.646,3.940e-19
1,LeadPortalActiveCount,3997,361,173.963,71.490,102.472,58.90%,30.115,4.302e-08
2,StandardEstimateCount,3997,361,1090.374,547.507,542.867,49.79%,18.325,1.902e-05
3,ServicedEstimateCount,3997,361,590.138,235.280,354.858,60.13%,16.562,4.793e-05
4,OnlinePaymentCountThisYear,3997,361,49.108,3.712,45.396,92.44%,16.381,5.269e-05
5,BudgetCount,3997,361,8.418,6.418,2.000,23.75%,16.007,6.416e-05
6,OnlinePaymentValueThisYear,3997,361,39496.135,1485.673,38010.462,96.24%,13.913,1.939e-04
7,TimesheetCount,3997,361,4754.244,2577.474,2176.771,45.79%,13.395,2.553e-04
8,InvoiceCount,3997,361,1455.899,568.590,887.309,60.95%,12.070,5.175e-04
9,ActiveBudgetCount,3997,361,3.268,3.094,0.174,5.33%,1.077,2.994e-01


## Segmented F-tests (ARR size + Persona)

We’ll only run segment-level tests when both groups have **at least 30 rows**:

- `n_active >= 30`
- `n_churned >= 30`

We start by checking segments of `ARR Category` and `Persona` separately, then run the `Persona × ARR Category` permutations.


In [23]:
# Basic cleanup for segmentation columns
for col in ["Persona", "ARR Category"]:
    if col in accounts_df.columns:
        accounts_df[col] = accounts_df[col].astype("string").str.strip().fillna("Missing")


def segment_counts(
    df: pd.DataFrame,
    segment_cols: list[str],
    group_col: str = "AccountStatusBinary",
) -> pd.DataFrame:
    """Counts of Active/Churned within each segment."""
    counts = (
        df.groupby(segment_cols + [group_col], dropna=False)
        .size()
        .unstack(group_col, fill_value=0)
        .reset_index()
    )

    # Ensure both columns exist even if a segment is missing one group
    if "Active" not in counts.columns:
        counts["Active"] = 0
    if "Churned" not in counts.columns:
        counts["Churned"] = 0

    counts["min_n"] = counts[["Active", "Churned"]].min(axis=1)
    return counts.sort_values(["min_n", "Active", "Churned"], ascending=False).reset_index(drop=True)


def valid_segments(
    counts_df: pd.DataFrame,
    min_n: int = 30,
) -> pd.DataFrame:
    """Filter to segments with sufficient sample size in both groups."""
    return counts_df[(counts_df["Active"] >= min_n) & (counts_df["Churned"] >= min_n)].copy()


MIN_N = 30
exclude = {"CASE ID", "ChurnFlag"}


In [24]:
# 1) Segment feasibility checks (run separately first)
arr_counts = segment_counts(accounts_df, ["ARR Category"])
persona_counts = segment_counts(accounts_df, ["Persona"])

print("ARR Category segments (counts):")
display(arr_counts)
print("\nValid ARR Category segments (min 30 per group):")
display(valid_segments(arr_counts, MIN_N))

print("\nPersona segments (counts):")
display(persona_counts)
print("\nValid Persona segments (min 30 per group):")
display(valid_segments(persona_counts, MIN_N))


ARR Category segments (counts):


AccountStatusBinary,ARR Category,Active,Churned,min_n
0,SMB,2058,266,266
1,MidMarket,1690,83,83
2,Enterprise,249,12,12



Valid ARR Category segments (min 30 per group):


AccountStatusBinary,ARR Category,Active,Churned,min_n
0,SMB,2058,266,266
1,MidMarket,1690,83,83



Persona segments (counts):


AccountStatusBinary,Persona,Active,Churned,min_n
0,P2,1153,117,117
1,P1,688,98,98
2,P3,1153,67,67
3,P4,289,23,23
4,G3,221,23,23
5,G2,259,20,20
6,G1,176,9,9
7,G4,58,4,4



Valid Persona segments (min 30 per group):


AccountStatusBinary,Persona,Active,Churned,min_n
0,P2,1153,117,117
1,P1,688,98,98
2,P3,1153,67,67


In [25]:
def run_segmented_f_tests(
    df: pd.DataFrame,
    segment_cols: list[str],
    min_n: int = 30,
    top_k_per_segment: int = 10,
    group_col: str = "AccountStatusBinary",
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """Run per-segment metric F-tests; returns (counts_df, results_long_df)."""
    counts_df = segment_counts(df, segment_cols, group_col=group_col)
    valid_df = valid_segments(counts_df, min_n)

    all_results: list[pd.DataFrame] = []
    for _, seg in valid_df.iterrows():
        mask = pd.Series(True, index=df.index)
        for col in segment_cols:
            mask &= df[col].astype("string").fillna("Missing").eq(seg[col])

        seg_df = df.loc[mask].copy()
        ranked = f_test_rank_metrics(
            seg_df,
            group_col=group_col,
            exclude_cols=exclude,
            min_n_per_group=min_n,
        )
        if ranked.empty:
            continue

        for col in segment_cols:
            ranked[col] = seg[col]

        ranked["segment_size_active"] = int(seg["Active"])
        ranked["segment_size_churned"] = int(seg["Churned"])

        all_results.append(ranked.head(top_k_per_segment))

    results_long = pd.concat(all_results, ignore_index=True) if all_results else pd.DataFrame()

    sort_cols = ["F", "p_value"]
    if not results_long.empty:
        results_long = results_long.sort_values(sort_cols, ascending=[False, True]).reset_index(drop=True)

    return counts_df, results_long


def show_top_per_segment(
    results_long: pd.DataFrame,
    segment_cols: list[str],
    k: int = 5,
) -> None:
    if results_long.empty:
        display(results_long)
        return

    out = (
        results_long.sort_values(["F", "p_value"], ascending=[False, True])
        .groupby(segment_cols, dropna=False)
        .head(k)
        .reset_index(drop=True)
    )

    display(
        out.style.format(
            {
                "mean_active": "{:.3f}",
                "mean_churned": "{:.3f}",
                "diff_active_minus_churned": "{:.3f}",
                "pct_diff_vs_active": "{:.2%}",
                "F": "{:.3f}",
                "p_value": "{:.3e}",
            }
        )
    )


In [26]:
# 2) Run F-tests by ARR Category (separate)
arr_counts_df, arr_results = run_segmented_f_tests(
    accounts_df,
    segment_cols=["ARR Category"],
    min_n=MIN_N,
    top_k_per_segment=10,
)

print("ARR Category: valid segments")
display(valid_segments(arr_counts_df, MIN_N))

print("ARR Category: top metrics per segment")
show_top_per_segment(arr_results, ["ARR Category"], k=5)


ARR Category: valid segments


AccountStatusBinary,ARR Category,Active,Churned,min_n
0,SMB,2058,266,266
1,MidMarket,1690,83,83


ARR Category: top metrics per segment


,metric,n_active,n_churned,mean_active,mean_churned,diff_active_minus_churned,pct_diff_vs_active,F,p_value,ARR Category,segment_size_active,segment_size_churned
0,LicenseCount,1690,83,16.617,8.928,7.689,46.27%,29.718,5.700e-08,MidMarket,1690,83
1,LicenseCount,2058,266,4.797,2.004,2.793,58.23%,28.967,8.103e-08,SMB,2058,266
2,StandardEstimateCount,1690,83,1335.627,358.506,977.121,73.16%,18.560,1.736e-05,MidMarket,1690,83
3,LeadPortalActiveCount,1690,83,244.089,82.241,161.848,66.31%,14.891,1.180e-04,MidMarket,1690,83
4,BudgetCount,1690,83,9.682,5.976,3.706,38.28%,12.404,4.393e-04,MidMarket,1690,83
5,TimesheetCount,1690,83,5944.825,2330.434,3614.392,60.80%,12.220,4.844e-04,MidMarket,1690,83
6,OnlinePaymentCountThisYear,2058,266,24.791,4.402,20.389,82.24%,5.755,1.652e-02,SMB,2058,266
7,TimesheetCount,2058,266,1558.507,2397.387,-838.880,-53.83%,4.549,3.305e-02,SMB,2058,266
8,OnlinePaymentValueThisYear,2058,266,13409.860,1833.583,11576.277,86.33%,3.757,5.271e-02,SMB,2058,266
9,LeadPortalActiveCount,2058,266,86.743,70.778,15.965,18.40%,1.366,2.426e-01,SMB,2058,266


In [27]:
# 3) Run F-tests by Persona (separate)
persona_counts_df, persona_results = run_segmented_f_tests(
    accounts_df,
    segment_cols=["Persona"],
    min_n=MIN_N,
    top_k_per_segment=10,
)

print("Persona: valid segments")
display(valid_segments(persona_counts_df, MIN_N))

print("Persona: top metrics per segment")
show_top_per_segment(persona_results, ["Persona"], k=5)


Persona: valid segments


AccountStatusBinary,Persona,Active,Churned,min_n
0,P2,1153,117,117
1,P1,688,98,98
2,P3,1153,67,67


Persona: top metrics per segment


,metric,n_active,n_churned,mean_active,mean_churned,diff_active_minus_churned,pct_diff_vs_active,F,p_value,Persona,segment_size_active,segment_size_churned
0,LicenseCount,1153,67,18.219,6.254,11.965,65.67%,44.128,4.624e-11,P3,1153,67
1,LicenseCount,1153,117,7.403,3.889,3.514,47.47%,36.315,2.197e-09,P2,1153,117
2,StandardEstimateCount,1153,117,717.473,295.521,421.951,58.81%,16.673,4.718e-05,P2,1153,117
3,BudgetCount,1153,117,8.529,5.803,2.726,31.96%,14.650,1.358e-04,P2,1153,117
4,LicenseCount,688,98,3.214,2.194,1.020,31.73%,11.966,5.711e-04,P1,688,98
5,TimesheetCount,1153,67,7419.098,3441.716,3977.382,53.61%,9.607,1.982e-03,P3,1153,67
6,StandardEstimateCount,1153,67,1693.903,858.313,835.589,49.33%,8.576,3.469e-03,P3,1153,67
7,LeadPortalActiveCount,1153,117,146.997,75.940,71.057,48.34%,8.398,3.821e-03,P2,1153,117
8,OnlinePaymentValueThisYear,1153,117,27369.168,600.111,26769.057,97.81%,8.144,4.392e-03,P2,1153,117
9,LeadPortalActiveCount,1153,67,249.252,101.642,147.610,59.22%,7.700,5.606e-03,P3,1153,67


In [28]:
# 4) Run F-tests by Persona × ARR Category permutations
perm_counts = segment_counts(accounts_df, ["Persona", "ARR Category"])
valid_perm = valid_segments(perm_counts, MIN_N)

print("Persona × ARR Category: valid permutations")
display(valid_perm)

perm_counts_df, perm_results = run_segmented_f_tests(
    accounts_df,
    segment_cols=["Persona", "ARR Category"],
    min_n=MIN_N,
    top_k_per_segment=10,
)

print("Persona × ARR Category: top metrics per permutation")
show_top_per_segment(perm_results, ["Persona", "ARR Category"], k=5)


Persona × ARR Category: valid permutations


AccountStatusBinary,Persona,ARR Category,Active,Churned,min_n
0,P1,SMB,600,93,93
1,P2,SMB,677,82,82
2,P3,SMB,301,42,42
3,P2,MidMarket,458,34,34


Persona × ARR Category: top metrics per permutation


,metric,n_active,n_churned,mean_active,mean_churned,diff_active_minus_churned,pct_diff_vs_active,F,p_value,Persona,ARR Category,segment_size_active,segment_size_churned
0,LicenseCount,677,82,4.981,1.939,3.042,61.07%,35.906,3.202e-09,P2,SMB,677,82
1,LicenseCount,600,93,2.620,1.817,0.803,30.64%,15.290,1.013e-04,P1,SMB,600,93
2,LicenseCount,301,42,7.053,2.381,4.672,66.24%,11.846,6.498e-04,P3,SMB,301,42
3,LeadPortalActiveCount,458,34,222.439,66.618,155.821,70.05%,8.254,4.244e-03,P2,MidMarket,458,34
4,StandardEstimateCount,677,82,601.712,317.841,283.871,47.18%,8.085,4.583e-03,P2,SMB,677,82
5,BudgetCount,458,34,9.068,5.559,3.509,38.70%,7.598,6.060e-03,P2,MidMarket,458,34
6,StandardEstimateCount,458,34,890.950,250.294,640.656,71.91%,7.537,6.265e-03,P2,MidMarket,458,34
7,InvoiceCount,458,34,1318.334,243.559,1074.775,81.53%,6.825,9.264e-03,P2,MidMarket,458,34
8,BudgetCount,677,82,8.174,5.890,2.284,27.94%,6.818,9.203e-03,P2,SMB,677,82
9,OnlinePaymentCountThisYear,600,93,26.432,0.699,25.733,97.36%,6.147,1.340e-02,P1,SMB,600,93


## Export results to `data/exports/`

This writes the key result tables back into the repo under `data/exports/` (handy for Tableau / sharing).


In [ ]:
# Find repo root by locating the `data/` folder
DATA_DIR = find_file(Path.cwd(), Path("data"))
EXPORT_DIR = DATA_DIR / "exports"
EXPORT_DIR.mkdir(parents=True, exist_ok=True)

# Overall
rank_df.to_csv(EXPORT_DIR / "f_test_overall_rank.csv", index=False)

# ARR Category
arr_counts_df.to_csv(EXPORT_DIR / "segment_counts_by_arr_category.csv", index=False)
valid_segments(arr_counts_df, MIN_N).to_csv(
    EXPORT_DIR / "valid_segments_by_arr_category.csv", index=False
)
arr_results.to_csv(EXPORT_DIR / "f_test_by_arr_category_rank_long.csv", index=False)

# Persona
persona_counts_df.to_csv(EXPORT_DIR / "segment_counts_by_persona.csv", index=False)
valid_segments(persona_counts_df, MIN_N).to_csv(
    EXPORT_DIR / "valid_segments_by_persona.csv", index=False
)
persona_results.to_csv(EXPORT_DIR / "f_test_by_persona_rank_long.csv", index=False)

# Persona × ARR Category
perm_counts_df.to_csv(
    EXPORT_DIR / "segment_counts_by_persona_x_arr_category.csv", index=False
)
valid_segments(perm_counts_df, MIN_N).to_csv(
    EXPORT_DIR / "valid_segments_by_persona_x_arr_category.csv", index=False
)
perm_results.to_csv(
    EXPORT_DIR / "f_test_by_persona_x_arr_category_rank_long.csv", index=False
)

EXPORT_DIR